## Classifying Dogs by Emotional expression using a CNN

### Imports

In [10]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass


import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # tensorflow INFO and WARNING messages are not printed 

import random 

import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import PIL
import PIL.Image
from PIL import Image, ImageOps
import tensorflow as tf

import matplotlib.pyplot as plt
import keras
from keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D

sns.set_context('notebook')
sns.set_style('white')

### Loading Images

In [ ]:
import pathlib
dataset_url = ""

# Download the data and track where it's saved using tf.keras.utils.get_file in a variable called data_dir
data_dir = keras.utils.get_file(origin=dataset_url,
                                   fname='dogs',
                                   untar=True)
data_dir = pathlib.Path(data_dir)
labels = list()

print(data_dir)
for folder in data_dir.glob('*'):
    labels.append(folder.name)
    print('The', folder.name, 'folder has', len(list(folder.glob('*.jpeg'))), 'pictures')
image_count = len(list(data_dir.glob('*/*.jpeg')))
print(image_count, 'total images')

### Viewing Image Examples

#### Angry

In [ ]:
content = list(data_dir.glob('angry/*'))
PIL.Image.open(str(content[1]))

#### Content

In [ ]:
content = list(data_dir.glob('content/*'))
PIL.Image.open(str(content[1]))

#### Disgusted

In [ ]:
disgusted = list(data_dir.glob('disgusted/*'))
PIL.Image.open(str(disgusted[1]))

#### Distressed

In [ ]:
distressed = list(data_dir.glob('distressed/*'))
PIL.Image.open(str(distressed[1]))

#### Excited

In [ ]:
excited = list(data_dir.glob('excited/*'))
PIL.Image.open(str(excited[1]))

#### Happy

In [ ]:
happy = list(data_dir.glob('happy/*'))
PIL.Image.open(str(happy[1]))

#### Loving

In [ ]:
loving = list(data_dir.glob('loving/*'))
PIL.Image.open(str(loving[1]))

#### Scared

In [ ]:
scared = list(data_dir.glob('scared/*'))
PIL.Image.open(str(scared[1]))

#### Shy

In [ ]:
shy = list(data_dir.glob('shy/*'))
PIL.Image.open(str(shy[1]))

### Processing Images

In [ ]:
# The batch size simply specifies the number of images to pass through our neural network at a time, until the entire training set is passed through. 32 is the default
batch_size = 32

# Here we set the size of all the images to be 200x200
img_height = 200
img_width = 200

train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "training",
  seed = 1234,
  image_size = (img_width, img_height),
  batch_size = batch_size,
  labels = "inferred",
  label_mode = "categorical",
  color_mode = "rgb"
)

valid = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "validation",
  seed = 1234,
  image_size = (img_height, img_width),
  batch_size = batch_size,
  labels = "inferred",
  label_mode = "categorical",
  color_mode = "rgb"
)

class_names = train.class_names
print(class_names)
num_classes = len(class_names)
print(num_classes)

first_batch = valid.take(1)

for img, lbl in first_batch:
    for i in np.arange(1):
        shape = img[i].shape

print(shape)

### Building and Training CNN

#### Simple CNN Model 1 w/ Adam Optimizer 50 Epochs

In [ ]:
# Let's build a CNN using Keras' Sequential capabilities

model_1 = Sequential()


## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=shape))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()
model_1.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer = 'adam', metrics = ['accuracy'])
model_1.fit(train, validation_data = valid, epochs = 50, batch_size = batch_size)

We still have 181K parameters, even though this is a "small" model.


#### Simple CNN Model 2 w/ RMSprop Optimizer 100 Epochs

In [ ]:
# Let's build a CNN using Keras' Sequential capabilities

model_2 = Sequential()


## 5x5 convolution with 2x2 stride and 32 filters
model_2.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=shape))
model_2.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_2.add(Conv2D(32, (5, 5), strides = (2,2)))
model_2.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_2.add(Flatten())
model_2.add(Dense(512))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

model_2.summary()

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_2.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model_2.fit(train, validation_data = valid, epochs = 100, batch_size = batch_size)

#### Advanced CNN Model

In [ ]:
# Let's build a CNN using Keras' Sequential capabilities

model_3 = Sequential()

model_3.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=shape))
model_3.add(Activation('relu'))
model_3.add(Conv2D(32, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Conv2D(64, (3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv2D(64, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512))
model_3.add(Activation('relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(num_classes))
model_3.add(Activation('softmax'))

model_3.summary()

# initiate RMSprop optimizer
opt_2 = keras.optimizers.RMSprop(lr=0.0005)

# Let's train the model using RMSprop
model_3.compile(loss='categorical_crossentropy', optimizer=opt_2, metrics=['accuracy'])
model_3.fit(train, validation_data = valid, epochs = 150, batch_size = batch_size)